In [11]:
from osman import Osman, OsmanConfig
from opensearchpy import OpenSearch
import pandas as pd
import re

In [12]:
OS_HOST_URL="http://localhost:9200"
TEMPLATE_NAME = 'semantic_search'
INDEX_NAME = "bible-nlp"
ID_KEY = "id"

In [13]:
host = 'localhost'
port = 9200

# Connect to OpenSearch
client = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    use_ssl=False,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False
)

In [14]:
os_man = Osman(OsmanConfig(host_url=OS_HOST_URL))

In [15]:
# Open the file and read its contents
with open('bible.txt', 'r') as file:
    bible_text = file.read()

In [16]:
# Split the text using regular expressions
verses = re.split(r'(\d+:\d+)', bible_text)

In [17]:
bible_dict = {}
# Iterate in steps of 2 starting from 1, since 0 is likely an empty string before the first verse
for i in range(1, len(verses), 2):
    verse_number = verses[i]
    verse_text = verses[i + 1].strip()  # Use strip() to remove any leading/trailing whitespace
    bible_dict[verse_number] = verse_text


In [18]:
def generate_id(verse_id):
    chapter, verse = verse_id.split(':')
    return int(chapter) * 1000 + int(verse)  # Convert chapter:verse into a unique integer ID

documents = [
    {"verse_id": verse_id, "content": content, "id": generate_id(verse_id)}
    for verse_id, content in bible_dict.items()
]


In [19]:
# Insert data into OpenSearch index
response = os_man.add_data_to_index(
    index_name=INDEX_NAME,
    documents=documents,
    id_key=ID_KEY,
    refresh=True  # Set to True for immediate availability, but be cautious in production
)
